## Homework 3: text analysis with transformers

Download and upzip the file named:  ef616c40-13c0-4212-b7cb-b3d3bbb08bff__2023_05_31T18_07_51.tsv.xz

You can find it in OLAT under the materials for the Session 13.

You will select part of the data, pre process it and apply a transformer for classification.

Source: https://swissdox.linguistik.uzh.ch/

In [2]:
import pandas as pd
#this dataset takes around 7min to laod, although you can start your code even if the whole file isn't loaded in colab environment
df=pd.read_csv('/content/ef616c40-13c0-4212-b7cb-b3d3bbb08bff__2023_05_31T18_07_51.tsv', sep='\t')
df.head()

,id,pubtime,medium_code,medium_name,rubric,regional,doctype,doctype_description,language,char_count,dateline,head,subhead,content_id,content
0,37375401,2019-07-08 00:00:00+02,NZZ,Neue Zürcher Zeitung,Front,NaN,PND,National daily newspaper,de,5637,NaN,Deutsche Bank mit radikalem Umbau,Umstrittene Investmentbank wird redimensioniert,000139ce-b726-82b2-f007-fb15cdb95418,<tx><ld><p>Mit einer tiefgreifenden Restruktur...
1,36234856,2019-01-23 00:00:00+01,NZZ,Neue Zürcher Zeitung,Sport,NaN,PND,National daily newspaper,de,4726,NaN,Europäische Basketballer werden in der weltbes...,Der 19-jährige Luka Doncic und der 40-jährige ...,0001b5d5-8813-bd8d-cd3b-cd82b8ea3afe,<tx><au>ROMAN SCHUPPLI</au><p>Vielleicht hänge...
2,39162757,2020-03-14 00:00:00+01,NZZ,Neue Zürcher Zeitung,Feuilleton,NaN,PND,National daily newspaper,de,8641,NaN,Was soll dieses Geschwurbel?,"Geht es um Literatur, wird das Fernsehen zum T...",0001d221-6383-4f0c-d818-17bf10ce294c,<tx><au>Rainer Moritz</au><p>Vielleicht sollte...
3,37863590,2019-09-21 00:00:00+02,NZZ,Neue Zürcher Zeitung,Schweiz,NaN,PND,National daily newspaper,de,212,NaN,RICHTIGSTELLUNG,NaN,0001f88a-0a85-53ee-51da-c969b6b42012,"<tx><p>Die Redaktion weist darauf hin, dass Re..."
4,35212980,2018-09-29 00:00:00+02,NZZ,Neue Zürcher Zeitung,Wirtschaft,NaN,PND,National daily newspaper,de,2202,Herausgegriffen,Wenn es Himmelsstürmern zu langweilig wird,NaN,00028240-09bd-09f1-b5ac-deb63840e86a,"<tx><p><au>Christian Steiner, Moskau</au>· Der..."


In [3]:
#See the unique rubrics available in the dataset
df.rubric.unique()

array(['Front', 'Sport', 'Feuilleton', 'Schweiz', 'Wirtschaft',
       'International', 'Briefe an die NZZ', 'Zürich und Region',
       'Reisen', 'Olympische Winterspiele', 'Meinung und Debatte',
       'Geldanlage', 'Literatur und Kunst', 'Panorama',
       'Börsen und Märkte', 'Wochenende', 'Reflexe', 'Unternehmenspraxis',
       'Gesellschaft', 'NZZ Domizil', 'Anlagefonds', 'Steuern sparen',
       'Finanzjahr', 'Fokus der Wirtschaft', 'Weltmeisterschaft',
       'Forschung und Technik', 'Mobilität'], dtype=object)

# Q1 (17.5 points)
A) Select the rubrics from: 'Schweiz','Zürich und Region', 'Meinung und Debatte'

B) Create two arrays, one with the text (cleaned) (this is ***text***) and another with the rubric name (this ***labels***)

In [4]:
#SOL
df1=df.loc[df['rubric'].isin(['Schweiz','Zürich und Region', 'Meinung und Debatte'])]

In [5]:
#SOL
import re
text=[]
labels=[]
for index, row in df.iterrows():
    y=''.join(str(row['rubric']))
    x=''.join([str(row['head']), str(row['subhead']), str(row['content'])])
    x=x.replace('nan', '.')
    x=re.sub(r"<(.*?)>", ".", x)
    text.append(x)
    labels.append(y)


In [6]:
#check their length
len(text), len(labels)



(222, 222)

In [7]:
#See the numebr of news pieces per class
import collections, numpy as np
counter = collections.Counter(labels)
counter

Counter({'Front': 6,
         'Sport': 19,
         'Feuilleton': 14,
         'Schweiz': 20,
         'Wirtschaft': 41,
         'International': 41,
         'Briefe an die NZZ': 3,
         'Zürich und Region': 17,
         'Reisen': 3,
         'Olympische Winterspiele': 1,
         'Meinung und Debatte': 13,
         'Geldanlage': 5,
         'Literatur und Kunst': 2,
         'Panorama': 7,
         'Börsen und Märkte': 10,
         'Wochenende': 1,
         'Reflexe': 5,
         'Unternehmenspraxis': 1,
         'Gesellschaft': 1,
         'NZZ Domizil': 1,
         'Anlagefonds': 1,
         'Steuern sparen': 1,
         'Finanzjahr': 1,
         'Fokus der Wirtschaft': 2,
         'Weltmeisterschaft': 2,
         'Forschung und Technik': 3,
         'Mobilität': 1})

#Q2 (17.5 points)
Create directories with the files in the right format for creating keras datasets:

swiss_dox_nzz/test/Meinung und Debatte

swiss_dox_nzz/test/Zürich und Region

swiss_dox_nzz/test/Schweiz

The same for train and val, for each class

In [8]:
#This code creates the file paths with the files in the right format for creating keras datasets
import os, pathlib, shutil, random
for class_ in ['Schweiz','Zürich und Region', 'Meinung und Debatte']:
  sample_text=[x for x,y in zip(text,labels) if y==class_]
  os.makedirs('swiss_dox_nzz' , exist_ok=True)
  main_path=pathlib.Path('swiss_dox_nzz')
  test_dir='test'
  train_dir='train'
  val_dir='val'
  os.makedirs(main_path / test_dir / class_, exist_ok=True)
  os.makedirs(main_path / train_dir / class_, exist_ok=True)
  os.makedirs(main_path / val_dir / class_, exist_ok=True)
  print(class_)

  np.random.shuffle(sample_text)
  ss=np.array_split(sample_text, 10)
  training, test, validate = ss[:7], ss[7:9], ss[9:]

  [np.savetxt(str(main_path)+'/'+test_dir+"/"+class_+"/sample"+str(np.random.randint(1000))+'.txt', x, newline=" ", fmt='%s') for x in test]
  [np.savetxt(str(main_path)+'/'+train_dir+"/"+class_+"/sample"+str(np.random.randint(1000))+'.txt', x, newline=" ", fmt='%s') for x in training]
  [np.savetxt(str(main_path)+'/'+val_dir+"/"+class_+"/sample"+str(np.random.randint(1000))+'.txt', x, newline=" ", fmt='%s') for x in validate]


Schweiz
Zürich und Region
Meinung und Debatte


In [9]:
sample_text

['Mangelnde Mobilität...Von KONRAD PAUL LIESSMANN..Ein heisser Sommer geht zu Ende, der Frühherbst kündigt sich an, die unerbittlichen Touristenströme in den historischen Stadtzentren ebben ab, die demütigenden Schlangen an den Flughäfen werden etwas kürzer, die unübersehbaren Staus auf den Autobahnen lockern sich, in den Fernzügen findet man vereinzelt wieder Sitzplätze, die Zelte werden abgebrochen, die Wohnwagen bilden nicht mehr endlose Kolonnen, die infernalisch dröhnenden Motorradfahrer verschwinden von den Alpenpässen, nur die grossen Kreuzfahrtschiffe lichten noch immer die Anker. Viele Menschen waren unterwegs in diesen Monaten und huldigten dem Gott der motorisierten Mobilität. Gleichzeitig aber nahm in diesem Sommer das Erschrecken über jenes Ansteigen der Temperaturen zu, das sich nicht zuletzt dieser Mobilität verdankt...Machen wir uns nichts vor: Theoretisch sind wir vom Klimawandel überzeugt und warnen vor seinen negativen Folgen, moralisch sorgen wir uns um die sich erw

#Q3 (17.5 points)
Create a dataset from the directory created

In [10]:
#SOL
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "swiss_dox_nzz/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "swiss_dox_nzz/val", batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "swiss_dox_nzz/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 21 files belonging to 3 classes.
Found 3 files belonging to 3 classes.
Found 6 files belonging to 3 classes.


In [11]:
#Data Transformation

from tensorflow.keras import layers
#resulting in a tensor of shape [batch_size, output_sequence_length], can be padded or truncated
max_length = 600
#The maximum size of the vocabulary for this layer.
max_tokens = 20000
#if output mode is 'tf-idf'
    #It weights a given term by taking “term frequency,”
    #how many times the term appears in the current document,
    #and dividing it by a measure of “document frequency,”
    #which estimates how often the term comes up across the dataset
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

#just like fit
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [12]:
for x,y in int_train_ds:
  print(x)
  print('nnn')
  print(y)
  break

tf.Tensor(
[[ 283 1501  582 ... 6940 1930   17]
 [   2  985 2406 ...    0    0    0]
 [4692 2854  104 ...  776 7059   30]
 ...
 [ 175  286   11 ... 4312   18 3615]
 [6296 6404   20 ...   44   36 1715]
 [  17   36  100 ...   60   45  666]], shape=(21, 600), dtype=int64)
nnn
tf.Tensor([2 0 1 2 0 1 0 0 0 1 0 2 1 2 2 1 1 0 2 2 1], shape=(21,), dtype=int32)


#Q4 (17.5 points)
Create a model baseline

In [13]:
#SOL
max_length=600
hidden_dim=16

inputs = keras.Input(shape=(max_length,))
x = layers.Dense(hidden_dim, activation="relu")(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_baseline = keras.Model(inputs, outputs)
model_baseline.compile(optimizer="adam",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])

model_baseline.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("baseline_text.keras",
                                    save_best_only=True)
]

#takes around 1h for the 10 epochs with the free colab version
model_baseline.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callbacks)
model_baseline = keras.models.load_model("baseline_text.keras")
print(f"Test acc: {model_baseline.evaluate(int_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600)]             0         
                                                                 
 dense (Dense)               (None, 16)                9616      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 51        
                                                                 
Total params: 9,667
Trainable params: 9,667
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 198ms/step - loss: 1005.8701 - accuracy: 0.5000
Test acc: 0.500


#Q5 (17.5 points)
Create a Transformer encoder (you might need to define the classes: TransformerEncoder, PositionalEmbedding)

In [14]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [16]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
#order
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
#context
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["sparse_categorical_accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

#Q6 (17.5 points)
Apply the model to predict the class of 3 newspieces, one from 'Reisen', one from 'Wochenende', and one from'Sport' rubric categories.

Compare the accuracy results for the 3, having in mind the algorithm capabilities.

Deliver

In [17]:
#HOW TO DELIVER: https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab
#download this notebook after all your answers and upload it to the files tab on the left
#right click on it to check the path
#replace the path ('/content/Untitled0.ipynb') by your file path
%%shell
jupyter nbconvert --to html /content/Untitled0.ipynb

#refresh the folder (not the page)
#download the html

[NbConvertApp] WARNING | pattern '/content/Untitled0.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--exec

CalledProcessError: ignored